In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
from scipy.stats import norm
import japanize_matplotlib
from scipy.stats import shapiro

In [ ]:
file_path = 'AdvanceData.xlsx'  # Excelファイル名
target_column = '調和性'         # 列名

df = pd.read_excel(file_path)

data = df[target_column].dropna()
print(f"データ読み込み完了: {len(data)} 件")
print(f"基本統計量:\n{data.describe()}")

In [ ]:
plt.figure(figsize=(8, 6))

stats.probplot(data, dist="norm", plot=plt)

plt.title(f"Q-Qプロット: {target_column}")
plt.xlabel("理論上の分位数 (正規分布)")
plt.ylabel("実際のデータ値")
plt.grid(True)
plt.show()

In [ ]:
# 1. データからパラメータ（平均と標準偏差）を推定する
mu, std = norm.fit(data)

# 2. グラフの設定
plt.figure(figsize=(10, 6))

# ヒストグラム
plt.hist(data, bins='auto', density=True, alpha=0.6, color='skyblue', label='実際のデータ')

# 正規分布曲線（理論値)
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)

# 曲線をプロット
plt.plot(x, p, 'r', linewidth=2, label=f'正規分布曲線\n(平均={mu:.2f}, SD={std:.2f})')

# 3. 装飾
plt.title(f'{target_column} の分布と正規分布曲線の比較')
plt.xlabel('値')
plt.ylabel('確率密度')
plt.legend()
plt.grid(True, alpha=0.5)

plt.show()

In [ ]:
# 歪度と尖度の計算
skewness = df.skew(numeric_only=True)
kurtosis = df.kurt(numeric_only=True)

result = pd.DataFrame({
    '歪度 (Skewness)': skewness,
    '尖度 (Kurtosis)': kurtosis
})

print("--- 分析結果 ---")
display(result)

In [ ]:
def diagnose_skewness(data, column_name):
    """
    データの歪度（Skewness）を計算し、診断結果を表示・描画する関数
    """
    # 1. 歪度の計算
    skew_val = stats.skew(data)
    
    # 2. 判定
    if abs(skew_val) < 0.5:
        judgment = "ほぼ対称（正規分布に近い）"
    elif skew_val >= 0.5:
        judgment = "正の歪み（右に裾が長い）"
    else: # skew_val <= -0.5
        judgment = "負の歪み（左に裾が長い）"

    print(f"--- 診断結果: {column_name} ---")
    print(f"歪度 (Skewness): {skew_val:.4f}")
    print(f"判定: {judgment}")
    
    # 3. 可視化
    plt.figure(figsize=(8, 5))
    sns.histplot(data, kde=True, color='skyblue')
    plt.title(f"{column_name} の分布\n(歪度: {skew_val:.2f} / {judgment})")
    plt.axvline(x=np.mean(data), color='red', linestyle='--', label='平均値')
    plt.axvline(x=np.median(data), color='green', linestyle='-', label='中央値')
    plt.legend()
    plt.grid(axis='y', alpha=0.5)
    plt.show()

In [ ]:
diagnose_skewness(data, f"{target_column}")

In [ ]:
def check_normality(series, col_name):
    """
    シャピロ・ウィルク検定を行い、正規性を判定する関数
    """
    # 欠損値を除去して実行（欠損があるとエラーになる場合があります）
    clean_series = series.dropna()
    
    # 検定の実行
    stat, p = shapiro(clean_series)
    
    print(f"--- 分析対象: {col_name} ---")
    print(f"統計量 W : {stat:.4f}")
    print(f"p値      : {p:.4f}")
    
    # 判定 (有意水準 0.05)
    # p > 0.05 の場合、正規分布であるという帰無仮説を棄却できないため
    # 「正規分布に従っていないとは言えない（＝正規分布とみなす）」と判断します。
    if p > 0.05:
        print("判定     : 正規分布である可能性が高い (p > 0.05)")
    else:
        print("判定     : 正規分布ではない可能性が高い (p <= 0.05)")
    print()

# 各列で実行
check_normality(data, f"{target_column}")

In [ ]:
print("欠損率 (%)")
# data の欠損率を計算
missing_rate = data.isnull().sum() / len(data) * 100
print(missing_rate)

print("\n外れ値の数 (IQR法)")
# data に対してIQR法を適用
# 第1四分位数(Q1)と第3四分位数(Q3)
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

# 外れ値の判定 (True/False のDataFrameになります)
outliers_mask = ((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR)))

# 各列の外れ値の合計数を表示
outliers_count = outliers_mask.sum()
print(outliers_count)

In [ ]:
# 1. 分布診断
skew_values = df[['UWES_ave', 'BAT_ave', 'PERMA']].skew()
print("Skew values:\n", skew_values)

# 2. 非正規項目をWinsorize（scipy.stats を既に import 済みなので stats.mstats.winsorize を使用）
for col in skew_values[abs(skew_values) > 2].index:
    df[f'{col}_robust'] = stats.mstats.winsorize(df[col], limits=[0.05, 0.05])

# 3. 相関行列（ピアソン）: 実在する列のみを選択
cols_for_corr = ['UWES_ave', 'BAT_ave', 'PERMA']
corr_matrix = df.filter(cols_for_corr).corr()


In [ ]:
df_non_normal = df[['UWES_ave', 'BAT_ave']] 
spearman_corr = df_non_normal.corr(method='spearman')
print("Spearman:\n", spearman_corr)